In [162]:

monto=3000
plz=20
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import urllib3
import json
import requests 
import time
import random

url = 'https://servicios.cmfchile.cl/simuladorhipotecario/aplicacion?indice=101.2.3 \
&maxuf=20000&minuf=100&maxpeso=652419400&minpeso=3262097&paso=2&template=entidades&tipomoneda=1 \
&monto=@MONTO_$&tipocredito=3&tipotasa=1&plazo=@PLAZO_$&inst=OK& \
marcados=1&marcados=9&marcados=12&marcados=14&marcados=16&marcados=28&marcados=37&marcados=39&marcados=51&marcados=672'

url=url.replace('@PLAZO_$',str(int(plz))).replace('@MONTO_$',str(int(monto)))


session=requests.Session()
session.verify=False

r = session.get( url.replace(' ',''), headers={
'Upgrade-Insecure-Requests': '1',
'Connection': 'keep-alive',
'Content-Type': 'application/x-www-form-urlencoded', \
'Referer': 'https://servicios.cmfchile.cl/simuladorhipotecario/aplicacion?indice=101.2.1', \
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
root = lxml.html.fromstring(r.text)

#test cuantos bancos en el listado 
nbancos=len(root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr"))
aux1=[];
for i in range(1,nbancos):
    banco=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[0].text
    tipo=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[1].text
    dividendo=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[2].text
    dividendo=dividendo.replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.').replace('\t','').replace(' ','')
    moneda=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[3].text
    tipotasa=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[4].text
    tasa=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[5].text
    tasa=tasa.replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.').replace('\t','').replace(' ','').replace('%','')
    
    CAE=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[6].text
    CAE=CAE.replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.').replace('\t','').replace(' ','').replace('%','')
    
    aux1=aux1+[[banco,tipo,dividendo,moneda,tipotasa,tasa,CAE]]
aux1=pd.DataFrame(aux1,columns=['banco','tipo','dividendo','moneda','tipotasa','tasa','CAE'])
## part1 
a=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[1]/ul[1]/li[contains(@class,'list-group-item')]")
aux=[]
for i in range(0, len(a)):
    b=a[i].text
    c=b.split(' ')
    d=[c[len(c)-1].replace('\xa0','').replace('\n','').replace(',','.')]
    aux=aux+[float(c[len(c)-1].replace('\xa0\n','').replace(',','.'))]
e=pd.DataFrame(np.array(aux).reshape(int(len(aux)/3),3),columns=['Etitulos','GastoNotarial','TasacionUF'])
aux1=aux1.join(e)

a=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[2]/ul[1]/li[contains(@class,'list-group-item')]")
aux=[]
for i in range(0, len(a)):
    b=a[i].text
    c=b.split(' ')
    d=c[len(c)-1].replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.')
    aux=aux+[float(d)]
e=pd.DataFrame(np.array(aux).reshape(int(len(aux)/2),2),columns=['divSinSeg_UF','divSinSeg_clp'])
aux1=aux1.join(e)

a=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[3]/ul[1]/li[contains(@class,'list-group-item')]")
aux=[]
for i in range(0, len(a)):
    b=a[i].text
    c=b.split(' ')
    d=c[len(c)-1].replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.')
    aux=aux+[float(d)]
e=pd.DataFrame(np.array(aux).reshape(int(len(aux)/2),2),columns=['SegDesgravamenUF','SegDesgravamenCLP'])
aux1=aux1.join(e)

a=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[4]/ul[1]/li[contains(@class,'list-group-item')]")
aux=[]
for i in range(0,len(a)):
    if i/2!=int(i/2):
        b=a[i].text
        c=b.split(' ')
        d=c[len(c)-1].replace('\xa0','').replace('Chile','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.')
        aux=aux+[d]
e=pd.DataFrame(np.array(aux).reshape(int(len(aux)/1),1),columns=['fecha Actualizacion'])
aux1=aux1.join(e)

a=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[3]/ul[2]/li[contains(@class,'list-group-item')]")
aux=[]
for i in range(0, len(a)):
    b=a[i].text
    c=b.split(' ')
    d=c[len(c)-1].replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.')
    aux=aux+[float(d)]
e=pd.DataFrame(np.array(aux).reshape(int(len(aux)/2),2),columns=['SegIncendioUF','SegIncendioCLP'])
aux1=aux1.join(e)

a=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[3]/ul[3]/li[contains(@class,'list-group-item')]")
aux=[]
for i in range(0, len(a)):
    b=a[i].text
    c=b.split(' ')
    d=c[len(c)-1].replace('\xa0','').replace('$','').replace('UF','').replace('\n','').replace('.','').replace(',','.')
    aux=aux+[float(d)]
e=pd.DataFrame(np.array(aux).reshape(int(len(aux)/2),2),columns=['divConSeg_UF','divConSeg_clp'])
aux1=aux1.join(e)
aux1['monto_simulado']=monto
aux1['plazo_simulado']=plz
aux1['producto']='SIMULADOR_SBIF'    
aux1




c:\Users\sullo\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'servicios.cmfchile.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,banco,tipo,dividendo,moneda,tipotasa,tasa,CAE,Etitulos,GastoNotarial,TasacionUF,...,SegDesgravamenUF,SegDesgravamenCLP,fecha Actualizacion,SegIncendioUF,SegIncendioCLP,divConSeg_UF,divConSeg_clp,monto_simulado,plazo_simulado,producto
0,Banco del Estado de Chile,Mutuo no Endosable,629911,UF,Fija,3.85,4.80,2.52,2.50,2.25,...,0.336,10961.0,20/05/2022,1.132,36927.0,19.31,629911.0,3000,20,SIMULADOR_SBIF
1,Banco Bice,Mutuo no Endosable,630890,UF,Fija,4.70,4.82,6.20,3.20,3.50,...,0.108,3523.0,17/05/2022,0.084,2740.0,19.34,630890.0,3000,20,SIMULADOR_SBIF
2,Scotiabank Chile,Mutuo no Endosable,638719,UF,Fija,4.86,4.98,5.60,3.00,2.25,...,0.183,5970.0,18/05/2022,0.000,0.0,19.58,638719.0,3000,20,SIMULADOR_SBIF
3,Coopeuch,Mutuo no Endosable,638719,UF,Fija,4.40,4.98,3.50,3.00,2.50,...,0.288,9395.0,26/05/2022,0.616,20095.0,19.58,638719.0,3000,20,SIMULADOR_SBIF
4,Itaú Corpbanca,Mutuo no Endosable,641981,UF,Fija,4.90,5.04,4.28,3.00,2.50,...,0.222,7242.0,05/05/2022,0.000,0.0,19.68,641981.0,3000,20,SIMULADOR_SBIF
5,Banco de Crédito e Inversiones,Mutuo no Endosable,641654,UF,Fija,4.80,5.05,2.98,2.64,2.11,...,0.291,9493.0,02/05/2022,0.081,2642.0,19.67,641654.0,3000,20,SIMULADOR_SBIF
6,Banco Santander-Chile,Mutuo no Endosable,649157,UF,Fija,4.95,5.18,3.50,3.50,2.28,...,0.363,11841.0,16/05/2022,0.000,0.0,19.90,649157.0,3000,20,SIMULADOR_SBIF
7,Banco de Chile,Mutuo no Endosable,649157,UF,Fija,4.77,5.18,4.00,3.20,2.50,...,0.264,8612.0,04/05/2022,0.384,12526.0,19.90,649157.0,3000,20,SIMULADOR_SBIF
8,Banco Internacional,Mutuo no Endosable,658617,UF,Fija,5.03,5.36,8.50,3.00,2.50,...,0.303,9884.0,23/05/2022,0.228,7438.0,20.19,658617.0,3000,20,SIMULADOR_SBIF


In [159]:
root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(1) + "]/td")[6].text

'4,80 % \n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t'